In [1]:
import os
import re
import numpy as np
import pandas as pd
import gensim
import collections
from nltk.corpus import stopwords
import nltk

import json

from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists, drop_database
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

/home/clutch/anaconda3/envs/insight/lib/python3.7/site-packages/sqlalchemy/sql/functions.py:68: SAWarning: The GenericFunction 'array_agg' is already registered and is going to be overriden.
  "is going to be overriden.".format(identifier))


In [2]:
# using function from "Keeping API Keys Secret.ipynb"
# by https://github.com/dylburger
# def get_file_contents(filename):
#     """ 
#     Given a filename,
#     return the contents of that file
#     """
#     try:
#         with open(filename, 'r') as f:
#             # assumed file is a single line with key
#             return f.read().strip()
#     except FileNotFoundError:
#         print("'%s' file not found" % filename)

In [3]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
# dbname = 'insight_db'
# username = 'postgres' # change this to your username
# password = get_file_contents('../keys/psql_key')

In [4]:
# 'engine' is a connection to a database
# engine = create_engine('postgres://%s:%s@localhost/%s'%(username,password,dbname))
# # load sql_magic so we can write SQL in Jupyter Notebooks
# %load_ext sql_magic

# # setup SQL connection to the postgreSQL engine we created
# %config SQL.conn_name = 'engine'
# print(engine.url)

In [5]:
# connect to make queries using psycopg2
# con = psycopg2.connect(database = dbname, host = 'localhost', user = username, password = password)
# con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

### Import packages to perform Text Summarization on Youtube Video Comments

In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [7]:
# build a List of btopwords
stopwords = list(STOP_WORDS)

In [8]:
# sql query to grab all comments and store in dataframe
# sql = "SELECT * \
#        FROM comments"

In [9]:
# data_df = pd.read_sql(sql, con=con)

In [10]:
from_csv_filename = 'data/csv_files/original_data.csv'
data_df = pd.read_csv(from_csv_filename, sep=',')

In [ ]:
# merged comments by video_id
# comments_merged = pd.DataFrame(data_df.groupby('video_id').text.apply(lambda x: ", ".join(x)))

In [ ]:
# load spacy english language
nlp = spacy.load('en')

In [ ]:
# build an nlp document for video_id = K7tnCDXa-rY
video_doc_example = data_df[data_df.video_id=='P4D0sESi5So'].text.item()
nlp_obj = nlp(video_doc_example)

In [ ]:
# tokenization of text
mytokens = [token.text for token in nlp_obj]

In [ ]:
# build word frequency 
# word.text is tokenization in spacy
word_frequencies = {}
for word in nlp_obj:
    if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [ ]:
#word_frequencies

In [ ]:
# maximum word frequency
maximum_frequency = max(word_frequencies.values())

In [ ]:
for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

In [ ]:
# frequency table
#word_frequencies

In [ ]:
# sentence tokens
sentence_list = [sentence for sentence in nlp_obj.sents]

In [ ]:
# example of sentence tokenization, word tokenization and lowering All Text
# for t in sentence_list:
#     for w in t:
#         print(w.text.lower())
#[w.text.lower() for t in sentence_list for w in t]

In [ ]:
# Sentence Score via comparrng each word with sentence
sentence_scores = {}  
for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

In [ ]:
# Sentence Score via comparrng each word with sentence
# Alternative Method
lowered_sentence_list = [w.text.lower() for t in sentence_list for w in t ]
lowered_sentence_scores = {}  
for sent in lowered_sentence_list:  
        for word in sent:
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        lowered_sentence_scores[sent] = word_frequencies[word]
                    else:
                        lowered_sentence_scores[sent] += word_frequencies[word]

In [ ]:
# sentence score table
#sentence_scores

### Find the Top N sentences with largest score

In [ ]:
from heapq import nlargest

In [ ]:
summarized_sentences = nlargest(15, sentence_scores, key=sentence_scores.get)

In [ ]:
summarized_sentences_cleaned = []
for ss in summarized_sentences:
    if ss.text.find(', ')==0:
        idx = ss.text.find(', ')
        new_str = ss[idx+1:].text.capitalize()
    else:
        new_str = ss.text.capitalize()
    new_str = ' '.join(new_str.split())
    summarized_sentences_cleaned.append(new_str) 
summarized_sentences_cleaned

In [ ]:
# convert sentences from spacy span to strings for joining entire sentence
final_sentences = []
for w in summarized_sentences_cleaned:
    for i, a in enumerate(w.text):
        if not a.isalpha() and a.isspace():
            break
    final_sentences.append(w.text[i:].capitalize())
    #print(w.text[i:].capitalize())
#    print(w.text)

In [ ]:
# list comprehension of sentences converted from spacy.span to strings
final_sentences = [ w.text for w in summarized_sentences ]

In [ ]:
summary = ' '.join(summarized_sentences_cleaned)

In [ ]:
summary = summary.replace(u'\xa0', u'')
summary = summary.replace(' , ', ' ')
summary = ' '.join(summary.split())

In [ ]:
summary

In [ ]:
len(summary)

In [ ]:
len(video_doc_example)

### Compare to Gensim Summarization Package

In [ ]:
from gensim.summarization import summarize

In [ ]:
gensim_summary = summarize(video_doc_example, ratio=0.009)

In [ ]:
len(gensim_summary)

In [ ]:
gensim_summary

### Functions to use in app.py for Text Summarization

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

# build a list of stopwords
stopwords = list(STOP_WORDS)

def text_summarizer(document):
    raw_text = document
    doc = nlp(raw_text)
    # build word frequency
    # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in doc:  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    # sentence tokens
    sentence_list = [ sentence for sentence in doc.sents ]

    # calculate sentence score and ranking
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    # find N largest
    summary_sentences = nlargest(5, sentence_scores, key=sentence_scores.get)
    
    final_sentences = []
    for w in summary_sentences:
        for i, a in enumerate(w.text):
            if a.isalpha():
                break
        final_sentences.append(w.text[i:].capitalize())
    
    summary = ' '.join(final_sentences)
    summary = summary.replace(u'\xa0', u'')
    print("Original Document\n")
    print(document)
    print("Total Length:",len(document))
    print('\n\nSummarized Document\n')
    print(summary)
    print("Total Length:",len(summary))

In [ ]:
text_summarizer(video_doc_example)